In [6]:
import os
import pickle
import argparse
import numpy as np
from model import Options, Seq2SeqAttn
import tensorflow as tf

In [2]:
# Parse the command line arguments.
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', type = str, default = '../pre-data/',
                    help = 'the directory to the data')

parser.add_argument('--word_embeddings_path', type = str, default = '../pre-data/word_embeddings.npy',
                    help = 'the directory to the pre-trained word embeddings')
parser.add_argument('--VAD_path', type = str, default = '../pre-data/VAD.npy',
                    help = 'the directory to VAD')
parser.add_argument('--tf_path', type = str, default = '../pre-data/tf.npy',
                    help = 'the directory to term frequency')
parser.add_argument('--VAD_loss_path', type = str, default = '../pre-data/VAD_loss.npy',
                    help = 'the directory to VAD loss for each word')
parser.add_argument('--ti_path', type = str, default = '../pre-data/mu_li.npy',
                    help = 'the directory to term importance')

parser.add_argument('--num_epochs', type = int, default = 1,
                    help = 'the number of epochs to train the data')
parser.add_argument('--batch_size', type = int, default = 64,
                    help = 'the batch size')
parser.add_argument('--learning_rate', type = float, default = 0.0001,
                    help = 'the learning rate')
parser.add_argument('--beam_width', type = int, default = 256,
                    help = 'the beam width when decoding')
parser.add_argument('--word_embed_size', type = int, default = 256,
                    help = 'the size of word embeddings')
parser.add_argument('--n_hidden_units_enc', type = int, default = 256,
                    help = 'the number of hidden units of encoder')
parser.add_argument('--n_hidden_units_dec', type = int, default = 256,
                    help = 'the number of hidden units of decoder')
# ? attn_depth
parser.add_argument('--attn_depth', type = int, default = 128,
                    help = 'attention depth')
parser.add_argument('--restore_path', type = str, default = '../model_dailydialog_rf',
                    help = 'the path to restore the trained model')
parser.add_argument('--restore_epoch', type = int, default = 0,
                    help = 'the epoch to restore')

parser.add_argument('--save_path', type = str, default = '../model_dailydialog_rf',
                    help = 'the path to save the trained model to')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
def read_data(data_path):
    def load_np_files(path):
        my_set = {}
        my_set['enc_input'] = np.load(os.path.join(path, 'enc_input.npy'))
        my_set['dec_input'] = np.load(os.path.join(path, 'dec_input.npy'))
        my_set['target'] = np.load(os.path.join(path, 'target.npy'))
        my_set['enc_input_len'] = np.load(os.path.join(path, 'enc_input_len.npy'))
        my_set['dec_input_len'] = np.load(os.path.join(path, 'dec_input_len.npy'))
        return my_set
    train_set = load_np_files(os.path.join(data_path, 'train'))
    valid_set = load_np_files(os.path.join(data_path, 'validation'))
    with open(os.path.join(data_path, 'token2id.pickle'), 'rb') as file:
        token2id = pickle.load(file)

    return train_set, valid_set, token2id

In [4]:
train_set, valid_set, token2id = read_data(args.data_path)
max_uttr_len_enc = train_set['enc_input'].shape[1]
max_uttr_len_dec = train_set['dec_input'].shape[1]

word_embeddings = np.load(args.word_embeddings_path)
VAD = np.load(args.VAD_path)
termfreq = np.load(args.ti_path) # term importance
termfreq = termfreq.reshape(-1,1)
VAD_loss = np.load(args.VAD_loss_path)
VAD_loss = VAD_loss.reshape(-1,1)


In [9]:
enc_input = train_set['enc_input'][0:5,:]
dec_input = train_set['dec_input'][0:5,:]

target = train_set['target'][0:5,:]
enc_input_len = tf.placeholder(tf.int32, shape = [args.batch_size])

In [12]:
with tf.variable_scope('embedding', reuse = tf.AUTO_REUSE):
    word_embeddings = tf.Variable(word_embeddings, name = 'word_embeddings')

    enc_input_embed = tf.nn.embedding_lookup(word_embeddings, enc_input)
    dec_input_embed = tf.nn.embedding_lookup(word_embeddings, dec_input)

    enc_input_VAD = tf.nn.embedding_lookup(VAD, enc_input)
    target_VAD = tf.nn.embedding_lookup(VAD, target)

    enc_input_tf = tf.nn.embedding_lookup(termfreq, enc_input)
    target_tf = tf.nn.embedding_lookup(termfreq, target)

    target_VAD_loss = tf.nn.embedding_lookup(VAD_loss, target)


In [30]:
init = tf.global_variables_initializer()
# _attention_Wb = tf.layers.Dense(units=3,use_bias=False)
# beta = _attention_Wb.apply(enc_input_embed)

with tf.Session() as sess:
    sess.run(init)
    v = sess.run(beta)
    print(v)  # will show you your variable.


[[[ 0.9983619  -0.2800465  -0.7723718 ]
  [ 0.31191275 -0.66652036  0.35526693]
  [ 0.7690006  -0.18002531 -0.11645786]
  [ 0.07688895  0.03892403 -0.02115805]
  [ 0.0986229   0.05810172 -0.032325  ]
  [ 0.5626453  -0.71000654  0.44308633]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]
  [ 0.15535344 -0.917687    1.0313559 ]]

 [[-0.08771887 -1.2519332   0.6506698 ]
  [ 1.3626304   0.02750129  0.03691274]
  [ 1.1154425   0.15062073  0.10992754]
  [ 0.03694773  0.00224326  0.00803591]
  [ 0.5626453  -0.71000654  0.44308633]
  [ 0.15535344 -0.917687    1.0313559 

In [31]:
num_examples = train_set['enc_input'].shape[0]
num_batches = num_examples // args.batch_size

In [32]:
num_batches

225

In [35]:
num_examples//256

56

In [41]:
train_set['enc_input'][:128,].shape

(128, 19)